### Import all the relavent modules

In [1]:
import os, sys
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import pickle
import math
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from moviepy.editor import *
from IPython.display import HTML
from scipy.ndimage.measurements import label

### Class to handle feature extraction

In [2]:
class CFeatures:
    """
    This class manages feature extraction for the given images.
    It provides 4 public API's, which can be called to get the 
    desired features.
    Public APIs ::
    1) convert_color
    2) color_hist_features
    3) bin_spatial_features
    4) get_hog_features
    """
    def __init__(self, color_space = 'BGR', spatial_size = (32,32), bins_range=(0,256), 
                 nbins = 32, orient = 9, pix_per_cell = 8, cell_per_block=2):
        """
        This is the constructor for the class CFeatures.
        It initializes all the member variables of this class.
        These member variables represent tunable hyper-parameters. 
        """
        self.color_space = color_space
        self.spatial_size = spatial_size
        self.bins_range = bins_range
        self.nbins = nbins
        self.orient = orient
        self.pix_per_cell = pix_per_cell
        self.cell_per_block = cell_per_block
        
    def convert_color(self, image):
        """
        This function converts the given image into the desired color space.
        
        @param[in] : image : Image for which color conversion is desired.
        @return color_converted_image : New image with converted colors.
        """
        if self.color_space != 'BGR':
            if self.color_space == 'HSV':
                color_converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            elif self.color_space == 'HLS':
                color_converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
            elif self.color_space == 'RGB':
                color_converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            elif self.color_space == 'YUV':
                color_converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
            elif self.color_space == 'YCrCb':
                color_converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        else: 
            color_converted_image = np.copy(image)

        return color_converted_image

    def color_hist_features(self, image):
        """
        This function extract color histogram features from a given image.
        
        @param[in] : image : Image for which features need to be extracted.
        @return hist_features : Histogram of color features.
        """

        # Compute the histogram of the 3 channels seprately
        channel_0 = np.histogram(image[:,:,0], self.nbins, self.bins_range)
        channel_1 = np.histogram(image[:,:,1], self.nbins, self.bins_range)
        channel_2 = np.histogram(image[:,:,2], self.nbins, self.bins_range)
        # Generating bin centers
        edges = channel_0[1]
        bin_centers = (edges[1:] + edges[0:len(edges)-1])/2
        # Concatenate the histograms into a single feature vector
        hist_features = np.concatenate((channel_0[0], channel_1[0], channel_2[0]))

        # Return the feature vector
        return hist_features
    
    def bin_spatial_features(self, image):
        """
        This function extracts spatial color features form a given image.

        @param[in] : image : Input image
        @return bin_spatial_color_features : Feature vector with spatially 
        binned feature vectors.
        """
        bin_spatial_color_features = cv2.resize(image, self.spatial_size).ravel() 
        # Return the feature vector
        return bin_spatial_color_features

    def get_hog_features(self, img, visualise=False, feature_vector=True):
        """
        This function extracts the HOG features from an image.
        
        @param[in] : img : input image
        @param[in] : visualise : Flag to return hog image.
        @param[in] : feature_vec : Flag to indicate the unrolling of feature vector.

        """
        if visualise:
            features, hog_image = hog(img, orientations=self.orient,
                              pixels_per_cell=(self.pix_per_cell, self.pix_per_cell), \
                              cells_per_block=(self.cell_per_block, self.cell_per_block), \
                              transform_sqrt=True, visualise=visualise, feature_vector=feature_vector)

            return features, hog_image

        else:
            features = hog(img, orientations=self.orient,
                              pixels_per_cell=(self.pix_per_cell, self.pix_per_cell), \
                              cells_per_block=(self.cell_per_block, self.cell_per_block), \
                              transform_sqrt=True, visualise=visualise, feature_vector=feature_vector)
            return features

### Class to handle the pipeline for training the classifier.

In [3]:
class CClassifier:
    """
    This class provides the method for training the classifier.
    It also saves the final model and the feature scaler to the disk.
    For training the classifier, the class provides 1 public API 
    and internally makes use of 4 private methods.
    
    Public API : train_classifier
    
    Private methods :
    1) __classifier
    2) __data_pre_process
    3) __augment_brightness
    4) __extract_features
    """
    def __init__(self, obj_features):
        """
        This is the constructor for the class CClassifier
        """        
        # Object of the CFeatures class
        self.obj_features = obj_features
    
    def train_classifier(self, not_cars, cars, aug_data=True):
        """
        This function executes the pipeline for training the classifier.
        It reads in the images, extracts the features, pre-processes the data and trains.
        After training, it saves the classifier on the disk.
        
        @param[in] not_cars : List of all the non car images
        @param[in] cars : List of the car images
        @param[in] aug_data : Flag to indicate if data augumentation should be done or not.
        """

        # Create a list to append feature vectors to
        not_car_features = []

        # Iterate through the list of car images
        for img in not_cars:
            # Read in each one by one in BGR format
            image = cv2.imread(img) 
            # Extract the features from the image
            not_car_features.append(self.__extract_features(image))
            if aug_data:
                aug_image = self.__augment_brightness(image)
                not_car_features.append(self.__extract_features(aug_image))

        # Create a list to append feature vectors to
        car_features = []

        # Iterate through the list of car images
        for img in cars:
            # Read in each one by one in BGR format
            image = cv2.imread(img)
            # Extract the features from the image
            car_features.append(self.__extract_features(image))
            if aug_data:
                aug_image = self.__augment_brightness(image)
                car_features.append(self.__extract_features(aug_image))

        # Define a labels vector based on features lists
        label_vector = np.hstack((np.ones(len(car_features)), np.zeros(len(not_car_features))))
        
        # Preprocess the data
        feature_scaler, X_train, X_test, y_train, y_test = \
                    self.__data_pre_process(car_features, not_car_features, label_vector)

        # Save feature scaler
        filename = 'feature_scaler.sav'
        pickle.dump(feature_scaler, open(filename, 'wb'))
        
        svc = self.__classifier(X_train, X_test, y_train, y_test)
        
        # Save the classifier
        filename = 'finalized_model.sav'
        pickle.dump(svc, open(filename, 'wb'))
            
    def __classifier(self, X_train, X_test, y_train, y_test):
        """
        This function creates a classifier and subsequently trains it.
        
        @param[in] : X_train : Training data
        @param[in] : X_test : Test data
        @param[in] : y_train : Trainign labels
        @param[in] : y_test : Test labels
        @return : svc : Final model
        """
        # Build a linear SVM classifier
        svc = LinearSVC()
        # Check the training time for the SVC
        t=time.time()
        svc.fit(X_train, y_train)
        t2 = time.time()
        print(round(t2-t, 2), 'Seconds to train SVC...')
        # Check the score of the SVC
        print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
        # Check the prediction time for a single sample
        t=time.time()
        n_predict = 10
        print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
        print('For these',n_predict, 'labels: ', y_test[0:n_predict])
        t2 = time.time()
        print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

        return svc
    
    def __data_pre_process(self, car_features, not_car_features, label_vector):
        """
        This function pre processes the training data. First it normalizes
        the training data then it shuffles it and splits into training and test sets.
        
        @param[in] : car_features
        @param[in] : not_car_features
        @param[in] : label_vector
        @return    :  feature_scaler, X_train, X_test, y_train, y_test
        """
        combined_feature_array = np.vstack((car_features, not_car_features)).astype(np.float64)                        
        feature_scaler = StandardScaler().fit(combined_feature_array)
        normalized_combined_feature_array = feature_scaler.transform(combined_feature_array)

        # Split up data into randomized training and test sets
        rand_state = 100
        X_train, X_test, y_train, y_test = train_test_split(
        normalized_combined_feature_array, label_vector, test_size=0.25, random_state=rand_state)

        return feature_scaler, X_train, X_test, y_train, y_test
    
    def __augment_brightness(self, image):
        """
        This function adds random brighteness to the input images
        
        @param[in] : image : Input image 
        @return : image1 : Augumented image.
        """
        image1 = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
        image1 = np.array(image1, dtype = np.float64)
        random_bright = .5+np.random.uniform()
        image1[:,:,2] = image1[:,:,2]*random_bright
        image1[:,:,2][image1[:,:,2]>255]  = 255
        image1 = np.array(image1, dtype = np.uint8)
        image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2BGR)
        return image1
    
    def __extract_features(self, image):
        """
        This function extracts spatial color features as well as HOG features from the image.
        
        @param[in] : image : Image from which features need to be extracted.
        @return : features : Vector containing all the features.
        """
        image = self.obj_features.convert_color(image)

        # Extract binned spatial color features
        spatial_features = self.obj_features.bin_spatial_features(image)

        # Extract features from color histogram
        hist_features = self.obj_features.color_hist_features(image)

        # Extract HOG features for all the channels
        hog_feature = []
        for channel in range(image.shape[2]):
            hog_feature_channel = self.obj_features.get_hog_features(image[:,:,channel])
            hog_feature.append(hog_feature_channel)

        hog_feature = np.ravel(hog_feature)

        # Append the new feature vector to the features list
        features = np.concatenate((spatial_features, hist_features, hog_feature))

        return features

### Class to handle vehicle detection pipeline

In [40]:
class CDetectionAttributes:
    """
    """
    def __init__(self):
        self.centroid = []
        self.x1y1 = []
        self.x2y2 = []
        self.num_detections = 0
        self.this_frame_detected = False
        self.num_not_detected = 0
        
class CVehicleDetection:
    """
    This class manages the pipeline for detecton of vehicle in a given image.
    It provides 1 public API, process_image(), for detecting vehicles. 
    Internally, it has 4 private methods, which help in different stages 
    of the detection pipeline (from implementing the core algorithm to drawing the boxes).
    
    Public API :: process_image
    
    Private methods ::
    1) __find_cars
    2) __add_heat
    3) __apply_threshold
    4) __draw_labeled_bboxes
    5) __iir_filter
    """
    
    def __init__(self, obj_features, scale = [1.5], ystart=[400], ystop=[655], 
                 xstart=[0], xstop=[1280], false_detection_threshold=2, 
                 iir_filter=True, cells_per_step=2, debug_plotting=False):
        """
        This is the constructor for the class CVehicleDetection.
        It initializes the hyper parameters required for detection.
        It also loads the classfier and the feature scale from the disk.
        """
        # Sanity checks
        assert len(scale)==len(ystart) and len(scale)==len(ystop) and len(ystart)==len(ystop), \
        "Length of sclae, ystart and ystop should be the same !"
        
        assert len(scale)==len(xstart) and len(scale)==len(xstop) and len(xstart)==len(xstop), \
        "Length of sclae, xstart and xstop should be the same !"
        
        self.obj_features = obj_features
        self.ystart = ystart
        self.ystop = ystop
        self.scale = scale
        self.xstart = xstart
        self.xstop = xstop
        self.false_detection_threshold = false_detection_threshold
        self.heat_map = np.zeros((720, 1280)).astype(np.float)
        self.car_detected_windows = []
        self.iir_filter = iir_filter
        self.cells_per_step = cells_per_step
        # Keep track of the vehicles detected
        self.detected_vehicle = []
        self.debug_plotting = debug_plotting
        
        # Load the saved model 
        filename = 'finalized_model.sav'
        with open(filename, mode='rb') as file:
            self.svc = pickle.load(file)

        # Load the saved feature scaler
        filename = 'feature_scaler.sav'
        with open(filename, mode='rb') as file:
            self.X_scaler = pickle.load(file)
            
    def process_image(self, image):
        """
        This function executes the pipeline for detecting a car on a single image.
        
        @param[in] : image : Input image
        @return : image with vehicles detected.
        """

        # Detect cars !
        self.__find_cars(image)
        
        #non_empty = [x for x in self.car_detected_windows if x != []]
        
        self.__add_heat()
        self.__apply_threshold()

        # Find final boxes from heatmap using label function
        labels = label(self.heat_map)
        
        window_img = np.copy(image)
        
        if self.iir_filter == False:
            window_img = self.__draw_labeled_bboxes(image, labels) 
            self.car_detected_windows = []
        else:
            window_img = self.__iir_filter(image, window_img, labels)
                    
        if self.debug_plotting:
            plt.imshow(self.heat_map, cmap='hot')
            plt.title("Heat Map")
            plt.show()
        
        # reset the heat map
        self.heat_map = np.zeros((720, 1280)).astype(np.float)
            
        return window_img

    
    def __iir_filter(self, image, window_img, labels):
        """
        This function filters out the detections to remove false positives
        and improve the bounding boxes of the detected ones by iir filtering
        @param[in] window_img : image to be drawn on.
        """
        
        colors = [[0,0,255],[0,255,0],[255,0,0],[255,255,0],[0,255,255],[255,0,255],[255,255,255]]
        filter_coeff = 0.7
        aspect_ratio = 1.5
        
        # Reset the detection flag for this frame
        for car in self.detected_vehicle:
            car.this_frame_detected = False

        for car_number in range(1, labels[1]+1):
            nonzero = (labels[0] == car_number).nonzero()
            # Identify x and y values of those pixels
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), 
                    (np.max(nonzerox), np.max(nonzeroy)))
            obj_detection = CDetectionAttributes()
            # Assuming each "box" takes the form ((x1, y1), (x2, y2))
            obj_detection.centroid = [int((bbox[0][0] + bbox[1][0]) / 2), 
                                      int((bbox[0][1] + bbox[1][1]) / 2)]
            obj_detection.x1y1 = [bbox[0][0] , bbox[0][1]]
            obj_detection.x2y2 = [bbox[1][0] , bbox[1][1]]

            # Iterate over the exsisting objects to find similar vehicles
            object_found = False

            for car in self.detected_vehicle:
                distance = math.hypot(car.centroid[0] - obj_detection.centroid[0], 
                                      car.centroid[1] - obj_detection.centroid[1])
                if distance < 50:
                    # Increment the detection count
                    car.num_detections += 1
                    car.num_not_detected = 0

                    # Filter the centroid to the mean of the centroids.
                    car.centroid[0] = int((filter_coeff*car.centroid[0] + (1-filter_coeff)*obj_detection.centroid[0]))
                    car.centroid[1] = int((filter_coeff*car.centroid[1] + (1-filter_coeff)*obj_detection.centroid[1]))

                    # Filter the edges
                    car.x1y1[0] = int((filter_coeff*car.x1y1[0] + (1-filter_coeff)*obj_detection.x1y1[0]))
                    car.x1y1[1] = int((filter_coeff*car.x1y1[1] + (1-filter_coeff)*obj_detection.x1y1[1]))

                    car.x2y2[0] = int((filter_coeff*car.x2y2[0] + (1-filter_coeff)*obj_detection.x2y2[0]))
                    car.x2y2[1] = int((filter_coeff*car.x2y2[1] + (1-filter_coeff)*obj_detection.x2y2[1]))

                    # Scale the box with proper aspect ratio
                    if (car.x2y2[1] - car.x1y1[1])*aspect_ratio < (car.x2y2[0] - car.x1y1[0]):
                        car.x2y2[0] = car.centroid[0] + int(((car.x2y2[1] - car.x1y1[1])*aspect_ratio)/2)
                        car.x1y1[0] = car.centroid[0] - int(((car.x2y2[1] - car.x1y1[1])*aspect_ratio)/2)

                    object_found = True
                    car.this_frame_detected = True
                    break

            if object_found == False:
                # Append the object to the list of detected objects if both sides are > 32 pixels
                if (abs(obj_detection.x2y2[0] - obj_detection.x1y1[0]) > 32) and \
                   (abs(obj_detection.x2y2[1] - obj_detection.x1y1[1]) > 32):

                        obj_detection.num_detections = 1
                        obj_detection.this_frame_detected = True
                        obj_detection.continuous_detection_count = 1
                        self.detected_vehicle.append(obj_detection)

        if len(self.car_detected_windows) == 5:
            self.car_detected_windows.pop(0)

        for itera,car in enumerate(self.detected_vehicle):
            # If the object was not found in this frame, reduce the detection count
            if car.this_frame_detected == False:
                car.num_detections -= 1
                car.num_not_detected += 1

            if car.num_not_detected == 7:
                self.detected_vehicle.pop(itera)
                break

            if car.num_detections > 5:
                window_img = cv2.rectangle(image, (car.x1y1[0], car.x1y1[1]), 
                                          (car.x2y2[0], car.x2y2[1]), colors[itera], 3)
        
        return window_img
        
    def __find_cars(self, image):
        """
        NOTE : This function is adapted from the Udacity lessons.
        This function implements a sliding window algorithm to extract the
        features from an image and then execute the classifier on it.
        @param[in] : image : Input image
        
        """
        car_detected_windows = []
        colors = [[0,0,255],[0,255,0],[255,0,0],[255,255,0],[0,255,255],[255,0,255],[255,255,255]]
        debug_plotting = False
        
        for iteration, _ in enumerate(self.scale):
            draw_image = np.copy(image)
            img_tosearch = image[self.ystart[iteration]:self.ystop[iteration],
                                 self.xstart[iteration]:self.xstop[iteration],:]
        
            ctrans_tosearch = self.obj_features.convert_color(img_tosearch)
        
            if self.scale[iteration] != 1:
                imshape = ctrans_tosearch.shape
                ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/self.scale[iteration]), \
                                                               np.int(imshape[0]/self.scale[iteration])))

            ch1 = ctrans_tosearch[:,:,0]
            ch2 = ctrans_tosearch[:,:,1]
            ch3 = ctrans_tosearch[:,:,2]

            # Define blocks and steps
            nxblocks = (ch1.shape[1] // self.obj_features.pix_per_cell) - self.obj_features.cell_per_block + 1
            nyblocks = (ch1.shape[0] // self.obj_features.pix_per_cell) - self.obj_features.cell_per_block + 1

            # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
            window = 64
            nblocks_per_window = (window // self.obj_features.pix_per_cell)- self.obj_features.cell_per_block + 1
            nxsteps = (nxblocks - nblocks_per_window) // self.cells_per_step + 1
            nysteps = (nyblocks - nblocks_per_window) // self.cells_per_step + 1

            # Compute individual channel HOG features for the entire image
            hog1 = self.obj_features.get_hog_features(ch1, feature_vector=False)
            hog2 = self.obj_features.get_hog_features(ch2, feature_vector=False)
            hog3 = self.obj_features.get_hog_features(ch3, feature_vector=False)

            for xb in range(nxsteps):
                for yb in range(nysteps):
                    ypos = yb*self.cells_per_step
                    xpos = xb*self.cells_per_step
                    # Extract HOG for this patch
                    hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                    hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                    hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                    hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

                    xleft = xpos*self.obj_features.pix_per_cell
                    ytop = ypos*self.obj_features.pix_per_cell

                    # Extract the image patch
                    subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))

                    # Get color features
                    spatial_features = self.obj_features.bin_spatial_features(subimg)
                    hist_features = self.obj_features.color_hist_features(subimg)

                    # Scale features and make a prediction
                    test_features = self.X_scaler.transform(np.hstack((spatial_features, \
                                                            hist_features, hog_features)).reshape(1, -1)) 
                    test_prediction = self.svc.predict(test_features)

                    if self.debug_plotting:
                        xbox_left = np.int(xleft*self.scale[iteration]) + self.xstart[iteration]
                        ytop_draw = np.int(ytop*self.scale[iteration])
                        win_draw = np.int(window*self.scale[iteration])
                        cv2.rectangle(draw_image, (xbox_left, ytop_draw + self.ystart[iteration]), 
                                  (xbox_left + win_draw, ytop_draw + win_draw + self.ystart[iteration]), 
                                  colors[iteration], 5)

                    if test_prediction == 1:
                        xbox_left = np.int(xleft*self.scale[iteration]) + self.xstart[iteration]
                        ytop_draw = np.int(ytop*self.scale[iteration])
                        win_draw = np.int(window*self.scale[iteration])
                        car_detected_windows.append(((xbox_left, ytop_draw + self.ystart[iteration]), 
                                          (xbox_left + win_draw, ytop_draw + win_draw + self.ystart[iteration])))
            if self.debug_plotting:
                plt.imshow(cv2.cvtColor(draw_image, cv2.COLOR_BGR2RGB))
                plt.show()
                
        self.car_detected_windows.append(car_detected_windows)
        
    def __add_heat(self):
        """
        This function generates a heat map based on the detected windows.
        """
        # Flatten the list containing the detected windows
        flat_list = [item for sublist in self.car_detected_windows for item in sublist]
        
        for box in flat_list:
            # Add += 1 for all pixels inside each bbox
            # Assuming each "box" takes the form ((x1, y1), (x2, y2))
            self.heat_map[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
            
    def __apply_threshold(self):
        """
        This function filters out the false positives.
        """
        # Zero out pixels below the threshold
        self.heat_map[self.heat_map <= self.false_detection_threshold] = 0
        
    def __draw_labeled_bboxes(self, img, labels):
        """
        This function draws the boxes on the detected window.
        @param[in] : img : Input image
        @return[in] : labels : pixels with the detected vehicles.
        @return : img : Image with boxes drawn
        """
        # Iterate through all detected cars
        for car_number in range(1, labels[1]+1):
            # Find pixels with each car_number label value
            nonzero = (labels[0] == car_number).nonzero()
            # Identify x and y values of those pixels
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            # Define a bounding box based on min/max x and y
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            # Draw the box on the image
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 5)
        # Return the image
        return img

### Helper function to read images from a given dir and its sub-dirs

In [41]:
def read_images(root_dir):
    """
    This function iterates over all the sub directories of a given root dir 
    and appends the names of the png images to a list.
    
    @param[in] : Path of the root dir.
    @return : image_data : List of names of all images. 
    """
    
    sub_dirs = [x[0] for x in os.walk(root_dir)]
    image_data = []
    for sub_dir in sub_dirs:
        image_path = os.path.join(root_dir, sub_dir)
        images = glob.glob(os.path.join(image_path,'*.png'))
        for image in images:
            image_data.append(image)
    
    return image_data

### Driver function to test the classifier

In [42]:
def test_classifier(image_path, obj_vehicle_detection):
    """
    This function tests the classifier on the test set of images.
    
    @param[in] : image_path : Path to the test images
    @param[in] : obj_vehicle_detection : Object of the vehicle detection class
    """
    test_images = os.listdir(image_path)

    # Loop over the dir containing test images and store the final images
    for image_name in test_images:
        image_path = os.path.join('test_images', image_name)
        image = cv2.imread(image_path)

        # Process the test image
        window_img = obj_vehicle_detection.process_image(image)
        
        # Display the final image with vehicles detected
        plt.imshow(cv2.cvtColor(window_img, cv2.COLOR_BGR2RGB))
        plt.title("Color Space == YCrCb, Cell Per Pixel == 8, Cells Per Block == 4, Orientations == 11")
        plt.show()
        output_image_path = os.path.join('output_test_images', 'final_'+ image_name )
        plt.imsave(output_image_path, cv2.cvtColor(window_img,cv2.COLOR_BGR2RGB))

### Driver function to process video frames

In [43]:
def process_video(video_name, obj_vehicle_detection):
    """
    This function processes a given test video by calling the 
    process_image function on every frame of the video.
    
    @param[in] : video_name : Name of the test video
    @param[in] : obj_vehicle_detection : Object of the vehicle detection class
    """
    clip = VideoFileClip(video_name)
    final_video = os.path.join('final_' + video_name)
    video_clip = clip.fl_image(obj_vehicle_detection.process_image)
    %time video_clip.write_videofile(final_video, audio=False)

    HTML("""
        <video width="960" height="540" controls>
        <source src="{0}">
        </video>
        """.format(final_video))

### Helper function for data exploration

In [44]:
%matplotlib inline

def data_exploration(not_cars, cars, obj_features):
    
    print("Total training samples for class not_cars == ", len(not_cars))
    print("Total training samples for class car == ", len(cars))
    
    # Visualize hog images
    car_image = cv2.imread(cars[100])
    features, hog_image = obj_features.get_hog_features(cv2.cvtColor(car_image, cv2.COLOR_BGR2GRAY), 
                                                            visualise=True)
    plt.subplot(1,2,1)
    plt.imshow(car_image, cmap='gray')
    plt.title("Original Car image")
    
    plt.subplot(1,2,2)
    plt.imshow(hog_image, cmap='gray')
    plt.title("Hog Features - YCrCb")
    plt.show()
    
    # Visualize some not cars samples from the data set

    fig, axes  = plt.subplots(nrows = 2, ncols = 2, figsize=(40,30))
    for row in axes:
        for col in row:
            rand_index = np.random.randint(len(not_cars))
            not_car_image = cv2.imread(not_cars[rand_index])
            col.imshow(not_car_image)
    
    fig, axes  = plt.subplots(nrows = 2, ncols = 2, figsize=(40,30))   
    for row in axes:
        for col in row:
            rand_index = np.random.randint(len(cars))
            car_image = cv2.imread(cars[rand_index])
            col.imshow(car_image)

### Main driver program to execute the pipeline

In [45]:
def main():
    """
    This is the main driver function to execute the entire vehicle detection
    pipeline.
    """
    
    # Read the images
    not_cars = read_images('C:\\Users\\rbahl\\CarND-Vehicle-Detection\\non-vehicles')
    cars = read_images('C:\\Users\\rbahl\\CarND-Vehicle-Detection\\vehicles')
    
    # Create an object for feature detection and set the hyper-parameters.
    obj_features = CFeatures(color_space = 'YCrCb', spatial_size = (32,32),
                            bins_range=(0,256), nbins = 32, orient = 9, 
                            pix_per_cell = 8, cell_per_block=4)
    
    # Data exploration
    data_exploration(not_cars, cars, obj_features)
    
    # Create an object to train and save the classifier.
    obj_classifier = CClassifier(obj_features)
    
    # Train the classifier.
    obj_classifier.train_classifier(not_cars, cars, aug_data=True)
    
    scale = [1.2, 1.5, 2.0]
    ystart = [400, 400, 400]
    ystop = [650, 680, 600]
    xstart= [550, 550, 550]
    xstop = [1280, 1280, 1280]


    # Create an object for vehicle detection
    obj_vehicle_detection = CVehicleDetection(obj_features, scale = scale, ystart=ystart, 
                                              ystop=ystop, xstart=xstart, xstop=xstop,
                                              false_detection_threshold=2,iir_filter=False, debug_plotting=False)
    
    # Create an object for vehicle detection in video (irr_filter ==True)
    obj_vehicle_detection_filter = CVehicleDetection(obj_features, scale = scale, ystart=ystart, 
                                              ystop=ystop, xstart=xstart, xstop=xstop, 
                                              false_detection_threshold=3,iir_filter=True)
    # Test the classifier on test images.
    image_path = 'test_images'
    test_classifier(image_path, obj_vehicle_detection)
    

    # Test the pipeline on the video
    video_name = 'project_video.mp4'
    process_video(video_name, obj_vehicle_detection_filter)

In [ ]:
main()